<a href="https://colab.research.google.com/github/durgesh077/AIAtt/blob/main/Blocks_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
try:
  cv2.imshow()
  show=cv2.imshow
except:
  from google.colab.patches import cv2_imshow as show

In [2]:
pxls=100
blackPxls=0.5

In [3]:
def preprocess(img,th=20):
  img=np.array(img,np.int32)
  shp=(400,400)
  for r in range(shp[0]):
    for c in range((img.shape[1])):
      if r ==0 and c==0:
        img[r][c]=img[r][c]>th
      elif r==0 and c:
        img[r][c]=img[r][c-1]+(img[r][c]>th)
      elif c==0 and r:
        img[r][c]=img[r-1][c]+(img[r][c]>th)
      else:
        img[r][c]=img[r-1][c]+img[r][c-1]-img[r-1][c-1]+(img[r][c]>th)
  for r in range(shp[0],img.shape[0]):
    for c in range((shp[1])):
      if r ==0 and c==0:
        img[r][c]=img[r][c]>th
      elif r==0 and c:
        img[r][c]=img[r][c-1]+(img[r][c]>th)
      elif c==0 and r:
        img[r][c]=img[r-1][c]+(img[r][c]>th)
      else:
        img[r][c]=img[r-1][c]+img[r][c-1]-img[r-1][c-1]+(img[r][c]>th)
  return img
def ratioBlack(img,r,c,h,w):
  h-=1
  w-=1
  sm=0
  if r ==0 and c==0:
    sm= img[h][w]
  elif r==0 and c:
    sm= img[h][c+w]- img[h][c-1]
  elif c==0 and r:
    sm= img[r+h][w]-img[r-1][w]
  else:
    sm= img[r+h][c+w]-img[r-1][c+w]-img[r+h][c-1]+img[r-1][c-1]
  total=((h+1)*(w+1))
  return 1-sm/total;

In [4]:
def convoluteShort(on,kernelShape):
  
  if on.shape[0]<200 or on.shape[1]<200:
    return 1

  y_len=kernelShape[0]
  x_len=kernelShape[1]
  y_from,x_from=0,0
  done=False
  for y in range(100):
    for x in range(100):
      res=ratioBlack(on,y,x,y_len,x_len)
      if res>blackPxls:
        y_from,x_from=(y+y_len,x+x_len)
        done=True
        break
    if done:
      break

  # try:
  while(ratioBlack(on,y_from,x_from,5,5)>blackPxls):
    x_from+=2
  x_from-=2
  while(1-ratioBlack(on,y_from,x_from,2,pxls)>blackPxls):
    y_from+=1
  return y_from,x_from
  # except:
  #   return 2


def nextHorizontalShort(on,y_prev,x_prev):
  y_prev+=30
  y_from=y_prev
  x_from=x_prev
  while(ratioBlack(on,y_from,x_from,5,pxls)>blackPxls):
      y_from+=1
  while(1-ratioBlack(on,y_from,x_from,5,pxls)>blackPxls):
      y_from+=1
  return y_from,x_from

def nextVerticalShort(on,y_prev,x_prev):
  x_prev+=30
  y_from=y_prev
  x_from=x_prev
  while(ratioBlack(on,y_from,x_from,pxls,5)>blackPxls):
    x_from+=1
  while(1-ratioBlack(on,y_from,x_from,pxls,5)>blackPxls):
      x_from+=1
  return y_from,x_from

In [5]:
def rotate(img,angle):
  rows = img.shape[0]
  cols = img.shape[1]

  img_center = (cols / 2, rows / 2)
  M = cv2.getRotationMatrix2D(img_center, angle, 1)

  rotated_image = cv2.warpAffine(img, M, (cols, rows),
                            borderMode=cv2.BORDER_CONSTANT,
                            borderValue=(255,255,255))
  return rotated_image

In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
def getAllCells(hori,ver,on):
  starts=[]
  for y in range(len(hori)-1):
    for x in range(len(ver)-1):
      leftTop=hori[y][0],ver[x][1]
      rightEnd=hori[y+1][0],ver[x+1][1]
      starts.append((leftTop,rightEnd))
  return starts

In [8]:
def giveHoriVer(marked):
  for ii in np.arange(0,5):
      for i in [ii,-(ii)]:
        global done
        done=False
        if i:
          timg=rotate(marked,i)
        else: 
          timg=marked
        img=preprocess(timg)
        try:
          leftEndpoint=convoluteShort(img,(30,30))
          horizontal=[leftEndpoint]
          vertical=[leftEndpoint]
          while(len(horizontal)<10):
            horizontal.append(nextHorizontalShort(img,horizontal[-1][0],horizontal[-1][1]))
          while(len(vertical)<8):
            vertical.append(nextVerticalShort(img,vertical[-1][0],vertical[-1][1]))
          done=True
          marked=timg
          break
        except:
          if ii==0:
            break
      if done :
        break
  # print(done)
  if not done:
    raise "not good image"
  else:
    return horizontal,vertical,marked

In [9]:
def extractImg(img,x1,y1,x2,y2):
  x1=x1-10
  y1=y1-10
  x2=x2+10
  y2=y2+10
  trimmed=img[y1:y2,x1:x2,np.newaxis]
  trimmed=tf.image.resize([trimmed],(224,224))
  trimmed=tf.image.grayscale_to_rgb(trimmed)
  return trimmed[0]

In [10]:
def getSubImgs(marked):
  horizontal,vertical,marked=giveHoriVer(marked)
  allBlocks=getAllCells(horizontal,vertical,marked)
  cells=marked.copy()
  subImgs=[]
  for lt,rb in allBlocks:
    cv2.rectangle(cells,lt[::-1],rb[::-1],100,2)
    subImgs.append(extractImg(marked,lt[1],lt[0],rb[1],rb[0]))
  subImgs=tf.convert_to_tensor(subImgs)
  return subImgs

In [11]:
def readImgGray(filename):
  imgGrid=cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
  return cv2.resize(imgGrid,(600,800),interpolation=cv2.INTER_AREA)

In [12]:
def preprocessForVGG(marked):
  subImgs=getSubImgs(marked)
  return vgg.preprocess_input(subImgs)

In [13]:
import tensorflow.keras.applications.vgg16  as vgg
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer,Flatten,Dropout,Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from tensorflow.keras import layers, models, Model, optimizers
from sklearn.model_selection import train_test_split as tts

In [14]:
%tensorflow_version 2.x
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.96.246.74:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.96.246.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.96.246.74:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [15]:
cellStates=['empty','present','absent']

In [16]:
subImgsEmpty=preprocessForVGG(readImgGray("grid2.png"))
empty=tf.constant(subImgsEmpty.shape[0]*[0],tf.uint8)

In [17]:
subImgsPresent=preprocessForVGG(readImgGray("excel.png"))
present=tf.constant(subImgsPresent.shape[0]*[1],tf.uint8)

In [18]:
subImgsAbsent=preprocessForVGG(readImgGray("excelA.png"))
absent=tf.constant(subImgsAbsent.shape[0]*[2],tf.uint8)

In [19]:
subImgs=tf.concat([subImgsEmpty,subImgsPresent,subImgsAbsent],0)
states=tf.concat([empty,present,absent],0)

In [20]:
# model.summary()

In [21]:
def giveModel():
  model=vgg.VGG16(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
  for layer in model.layers[:15]:
    layer.trainable=False
  x=model.output
  x=Flatten()(x)
  x=Dense(512,activation='relu')(x)
  x=Dropout(0.5)(x)
  x=Dense(256,activation='relu')(x)
  x=Dense(3,activation='softmax')(x)
  Attendance_model=Model(inputs=model.input,outputs=x)
  return Attendance_model

In [30]:
Attendance_model=giveModel()

In [31]:
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.6, patience=8, verbose=1, mode='max', min_lr=5e-5)
checkpoint = ModelCheckpoint('vgg16_finetune.h15', monitor= 'val_accuracy', mode= 'max', save_best_only = True, verbose= 1)
early_stop=EarlyStopping(monitor='val_loss')

In [32]:
learning_rate= 5e-5
Attendance_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=learning_rate), metrics=["accuracy"])

In [33]:
def createDataSet(X,y,is_training=False):
  ds=tf.data.Dataset.from_tensor_slices((X,y))
  ds=ds.batch(32).prefetch(32)
  if is_training:
    ds=ds.shuffle(32)
  return ds

In [34]:
X=subImgs.numpy()
y=states.numpy()

In [35]:
X_train,X_test,y_train,y_test=tts(X,y,test_size=0.25,random_state=42,shuffle=True)

In [36]:
Attendance_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [37]:
train_ds=createDataSet(X_train,y_train,True)
val_ds=createDataSet(X_test,y_test)

In [38]:
history = Attendance_model.fit(train_ds, epochs=15,validation_data=val_ds,callbacks=[lr_reduce])

Epoch 1/15
5/5 [==============================] - 5s 635ms/step - loss: 4.3960 - accuracy: 0.4752 - val_loss: 1.4231 - val_accuracy: 0.5625 - lr: 5.0000e-05
Epoch 2/15
5/5 [==============================] - 3s 537ms/step - loss: 0.6663 - accuracy: 0.7872 - val_loss: 0.4350 - val_accuracy: 0.8750 - lr: 5.0000e-05
Epoch 3/15
5/5 [==============================] - 3s 598ms/step - loss: 0.5790 - accuracy: 0.8298 - val_loss: 0.5265 - val_accuracy: 0.8542 - lr: 5.0000e-05
Epoch 4/15
5/5 [==============================] - 3s 533ms/step - loss: 0.4147 - accuracy: 0.8723 - val_loss: 0.2571 - val_accuracy: 0.8958 - lr: 5.0000e-05
Epoch 5/15
5/5 [==============================] - 3s 535ms/step - loss: 0.1268 - accuracy: 0.9504 - val_loss: 0.2101 - val_accuracy: 0.9167 - lr: 5.0000e-05
Epoch 6/15
5/5 [==============================] - 3s 540ms/step - loss: 0.0579 - accuracy: 0.9645 - val_loss: 0.2487 - val_accuracy: 0.9167 - lr: 5.0000e-05
Epoch 7/15
5/5 [==============================] - 3s 604ms

In [39]:
Attendance_model.evaluate(val_ds)

2/2 [==============================] - 1s 186ms/step - loss: 0.2624 - accuracy: 0.9167


[0.26239463686943054, 0.9166666865348816]

In [40]:
tf.argmax(Attendance_model.predict(subImgsEmpty),axis=1).numpy()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
weights=Attendance_model.get_weights()

In [43]:
import pickle

In [44]:
pickle.dump(weights,open("savedWeights/AttWeights.pkl","wb"))